## Imports

In [1]:
import os
import sys
sys.path.append('../')
from sklearn.model_selection import train_test_split

from TumorDetection.data.loader import DataPathLoader
from TumorDetection.data.dataset import TorchDataset
from TumorDetection.utils.dict_classes import DataPathDir, ReportingPathDir, Verbosity, Device
from TumorDetection.models.efsnet import EFSNet
from TumorDetection.models.utils.lightning_model import LightningModel
from TumorDetection.models.utils.trainer import Trainer

## Configuration params

In [2]:
MODEL_NAME = 'EFSNet_clf_seg'
DESCRIPTION = 'EFSNet with classification and binary segmentation.'
CLASS_WEIGHTS = [1., 3., 3.]
POS_WEIGHT = 5
FROM_CHECKPOINT = False
VALIDATE = True
TEST = True
VERBOSE = Verbosity.get('verbose')
DEVICE = Device.get('device')

## Path Finder

If not using DataPathLoader (*for BUSI Dataset*) consider passing paths as tuple of:  
- Image path
- List images masks paths associated
- List of associated label mask.

In [3]:
dp = DataPathLoader(DataPathDir.get('dir_path'))
paths = dp()

## Train Test Datasets

In [4]:
tr_paths, val_paths = train_test_split(paths, test_size=100, random_state=0, shuffle=True)
tr_td = TorchDataset(tr_paths)
val_td = TorchDataset(val_paths)

## Model definition.

In [5]:
lightningmodel = LightningModel(model=EFSNet(device=DEVICE,
                                            verbose=VERBOSE),
                                model_name=MODEL_NAME,
                                description=DESCRIPTION,
                                class_weights=CLASS_WEIGHTS,
                                pos_weight=POS_WEIGHT,
                                device=DEVICE)

## Training

In [7]:
trainer = Trainer(model_name=MODEL_NAME,
                  ckpt_dir=os.path.join(ReportingPathDir.get('dir_path'), 'ckpt'),
                  verbose=VERBOSE)
trainer(model=lightningmodel,
        train_data=tr_td,
        test_data=val_td,
        from_checkpoint=FROM_CHECKPOINT,
        validate_model=VALIDATE,
        test_model=TEST)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Torch is using cuda


You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Layer (type (var_name))                                           Input Shape               Output Shape              Param #                   Param %
LightningModel (LightningModel)                                   [1, 1, 256, 256]          [1, 1, 256, 256]          --                             --
├─EFSNet (model)                                                  [1, 1, 256, 256]          [1, 1, 256, 256]          --                             --
│    └─Encoder (encoder)                                          [1, 1, 256, 256]          [1, 64, 64, 64]           --                             --
│    │    └─InitialBlock (initial_block)                          [1, 1, 256, 256]          [1, 16, 128, 128]         166                         0.09%
│    │    └─DownsamplingBlock (downsampling_block1)               [1, 16, 128, 128]         [1, 64, 64, 64]           5,732                       3.04%
│    │    └─Sequential (factorized_blocks)                        [1, 64, 64, 64]       

Validation: |                                                                                                 …

ValueError: Target size (torch.Size([26, 2, 256, 256])) must be the same as input size (torch.Size([26, 1, 256, 256]))